# COVID19 - Brazil States

Install necessary packages for parallel computation:

```
pip install ipyparallel
ipcluster nbextension enable
pip install parallel-execute
```

To install for all users on JupyterHub, as root:
```
jupyter nbextension install --sys-prefix --py ipyparallel
jupyter nbextension enable --sys-prefix --py ipyparallel
jupyter serverextension enable --sys-prefix --py ipyparallel
```

start cluster at jupyter notebook interface

In [1]:
import urllib.request
import pandas as pd
import numpy as np

In [2]:
# Download data
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...


In [3]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-06-02,SE,TOTAL,state,7555,172,70,True,2298696.0,28.0,328.66460,0.0228
1,1,2020-06-01,SE,TOTAL,state,7233,166,69,False,2298696.0,28.0,314.65666,0.0230
2,2,2020-05-31,SE,TOTAL,state,6999,158,68,False,2298696.0,28.0,304.47697,0.0226
3,3,2020-05-30,SE,TOTAL,state,6805,149,67,False,2298696.0,28.0,296.03741,0.0219
4,4,2020-05-29,SE,TOTAL,state,6462,142,66,False,2298696.0,28.0,281.11590,0.0220
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,152116,2020-03-13,AL,TOTAL,state,1,0,5,False,3337357.0,27.0,0.02996,0.0000
2157,152117,2020-03-12,AL,TOTAL,state,1,0,4,False,3337357.0,27.0,0.02996,0.0000
2158,152118,2020-03-10,AL,TOTAL,state,1,0,3,False,3337357.0,27.0,0.02996,0.0000
2159,152119,2020-03-09,AL,TOTAL,state,1,0,2,False,3337357.0,27.0,0.02996,0.0000


In [4]:
# Model

In [5]:
states=['SP','ES','MG','RJ','PE','CE','AM']
states

['SP', 'ES', 'MG', 'RJ', 'PE', 'CE', 'AM']

'# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

In [6]:
#objective function Odeint solver
from scipy.integrate import odeint

#objective function Odeint solver
def lossOdeint(point, data, death, s_0, e_0, a_0, i_0, r_0, d_0, startNCases, ratioRecovered, weigthCases, weigthRecov):
    size = len(data)
    beta, beta2, sigma, sigma2, sigma3, gamma, b, mu = point
    def SEAIRD(y,t):
        S = y[0]
        E = y[1]
        A = y[2]
        I = y[3]
        R = y[4]
        D = y[5]
        p=0.2
        # beta2=beta
        y0=-(beta2*A+beta*I)*S-mu*S #S
        y1=(beta2*A+beta*I)*S-sigma*E-mu*E #E
        y2=sigma*E*(1-p)-gamma*A-mu*A #A
        y3=sigma*E*p-gamma*I-sigma2*I-sigma3*I-mu*I#I
        y4=b*I+gamma*A+sigma2*I-mu*R #R
        y5=(-(y0+y1+y2+y3+y4)) #D
        return [y0,y1,y2,y3,y4,y5]

    y0=[s_0,e_0,a_0,i_0,r_0,d_0]
    tspan=np.arange(0, size, 1)
    res=odeint(SEAIRD,y0,tspan) #,hmax=0.01)

    l1=0
    l2=0
    l3=0
    tot=0

    for i in range(0,len(data.values)):
        if data.values[i]>startNCases:
            l1 = l1+(res[i,3] - data.values[i])**2
            l2 = l2+(res[i,5] - death.values[i])**2
            newRecovered=min(1e6,data.values[i]*ratioRecovered)
            l3 = l3+(res[i,4] - newRecovered)**2
            tot+=1
    l1=np.sqrt(l1/max(1,tot))
    l2=np.sqrt(l2/max(1,tot))
    l3=np.sqrt(l3/max(1,tot))
    
    #weight for cases
    u = weigthCases  #Brazil US 0.1
    w = weigthRecov
    #weight for deaths
    v = max(0,1. - u - w)
    
    return u*l1 + v*l2 + w*l3

In [18]:
# Initial parameters
dfparam = pd.read_csv("data/param.csv")
dfparam = dfparam.dropna()
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-20,200.0,8000000.0,0.0001,0.0001,800.0,300.0,400.0,100.0,0.15,0.55,0.100
1,ES,2020-04-01,200.0,350000.0,0.0001,0.0001,50.0,10.0,25.0,150.0,0.15,0.65,0.001
2,MG,2020-04-01,200.0,2000000.0,0.0001,0.0001,100.0,100.0,50.0,100.0,0.05,0.55,0.100
3,RJ,2020-04-01,200.0,2000000.0,0.0001,0.0001,100.0,100.0,50.0,100.0,0.05,0.55,0.100
4,CE,2020-04-01,200.0,2000000.0,0.0001,0.0001,100.0,100.0,50.0,100.0,0.05,0.55,0.100
5,PE,2020-04-01,200.0,2000000.0,0.0001,0.0001,100.0,100.0,50.0,100.0,0.05,0.55,0.100
6,AM,2020-04-01,200.0,2000000.0,0.0001,0.0001,100.0,100.0,50.0,100.0,0.05,0.55,0.100


In [8]:
# Initial parameter optimization

In [9]:
import ray
ray.init()

# Load solver
GlobalOptimization = False
Yabox = False

if GlobalOptimization:
    import LearnerGlobalOpt as Learner  # basinhopping global optimization (several times minimize)
elif Yabox:
    import LearnerYabox as Learner #Yabox
else:
    import Learner as Learner #minimize

2020-06-04 13:25:28,158	WARNING resource_spec.py:163 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-06-04 13:25:28,159	INFO resource_spec.py:205 -- Starting Ray with 70.07 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-04 13:25:29,545	WARNING resource_spec.py:163 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2020-06-04 13:25:29,546	INFO resource_spec.py:205 -- Starting Ray with 70.07 GiB memory available for workers and up to 18.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


In [10]:
allStates=False
cleanRecovered=False
version="3"

results=[]
if allStates:
    for state in states:        
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        parameters
        learner = Learner.Learner.remote(state, lossOdeint, *parameters, cleanRecovered, version,version)
        #learner.train()
        #add function evaluation to the queue
        results.append(learner.train.remote())
else:
    state= "ES"
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    parameters
    cleanRecovered = False
    learner = Learner.Learner.remote(state, lossOdeint, *parameters, cleanRecovered, version)
    #learner.train()
    #add function evaluation to the queue
    results.append(learner.train.remote())

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

(pid=49562) RUNNING THE L-BFGS-B CODE
(pid=49562) 
(pid=49562)            * * *
(pid=49562) 
(pid=49562) Machine precision = 2.220D-16
(pid=49562)  N =            8     M =           10
(pid=49562) 
(pid=49562) At X0         3 variables are exactly at the bounds
(pid=49562)  The initial X is infeasible.  Restart with its projection.
(pid=49562) 
(pid=49562) At iterate    0    f=  1.29275D+04    |proj g|=  1.99000D-01
(pid=49562) 
(pid=49562) At iterate    1    f=  4.27688D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    2    f=  4.22498D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    3    f=  4.22398D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    4    f=  3.42322D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    5    f=  3.42312D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    6    f=  3.41481D+03    |proj g|=  5.00000D+01
(pid=49562) 
(pid=49562) At iterate    7    f=  3.40312D+03    

In [11]:
ray.shutdown()

(pid=49562) 
(pid=49562) At iterate   63    f=  5.50508D+02    |proj g|=  1.99995D-01
(pid=49562) 
(pid=49562)            * * *
(pid=49562) 
(pid=49562) Tit   = total number of iterations
(pid=49562) Tnf   = total number of function evaluations
(pid=49562) Tnint = total number of segments explored during Cauchy searches
(pid=49562) Skip  = number of BFGS updates skipped
(pid=49562) Nact  = number of active bounds at final generalized Cauchy point
(pid=49562) Projg = norm of the final projected gradient
(pid=49562) F     = final function value
(pid=49562) 
(pid=49562)            * * *
(pid=49562) 
(pid=49562)    N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
(pid=49562)     8     63    196     91     1     2   2.000D-01   5.505D+02
(pid=49562)   F =   550.507631390661     
(pid=49562) 
(pid=49562) CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
(pid=49562) 
(pid=49562)  Cauchy                time 0.000E+00 seconds.
(pid=49562)  Subspace minimization time 0.000E+00 s

# Plots

In [12]:
# %matplotlib inline 
%matplotlib widget
import matplotlib.pyplot as plt
import covid_plots

UsageError: unrecognized arguments: widget


In [13]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

In [14]:
selectedState='ES'

#select districts for plotting
states4Plot=['MG',
               'SP',
               'RJ',
               'CE',
               selectedState]

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#versio'n to identify the png file result
# version = "1"

#parameters for plotting
query = dfparam.query('state == "{}"'.format(state)).reset_index()
startdate = query['start-date'][0]
predict_range = query['prediction-range'][0]

#number of cases to start plotting model in log graph - real data = 100
startCase=100

In [15]:
#do not allow the scrolling of the plots

In [16]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

<IPython.core.display.Javascript object>

In [17]:
#plots one district or all districts

if allStates:
    for state in states:
        covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                                startCase, 5, version, show=True)
else: 
    covid_plots.covid_plots(state, states4Plot, startdate,predict_range, \
                                startCase, opt, version, show=True)

NameError: name 'covid_plots' is not defined